In [37]:
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import sys
import os
import gc
from datetime import date

from hmmlearn.hmm import GaussianHMM

In [39]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
os.chdir('/content/drive/My Drive/10708')
print("Current Working Directory: ", os.getcwdb())


%run Stock_Data.ipynb

Current Working Directory:  b'/content/drive/My Drive/10708'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed
(8315, 7)
                   Open         High          Low    Adj Close      Volume  Log Return       vol
Date                                                                                            
1990-01-02   353.399994   359.690002   351.980011   359.690002   162070000    0.000000       NaN
1990-01-03   359.690002   360.589996   357.890015   358.760010   192330000   -0.258889       NaN
1990-01-04   358.760010   358.760010   352.890015   355.670013   177000000   -0.865030       NaN
1990-01-05   355.670013   355.670013   351.350006   352.200012   158530000   -0.980414       NaN
1990-01-08   352.200012   354.239990   350.540009   353.790009   140110000    0.450431       NaN
...                 ...          ...          ...          ...         ...        

In [41]:
def get_most_probable_outcome(model, day_index):
    # Use the previous n_latency_days worth of data for predictions
    
    previous_data_start_index = max(0, 1)
    previous_data_end_index = max(0, day_index - 1)
    previous_data = data.iloc[
        previous_data_start_index:previous_data_end_index
    ]
    previous_data_features = data.iloc[day_index-1]

    outcome_score = []

    # Score all possible outcomes and select the most probable one to use for prediction
    for i in range(model.n_components):          
        total_data = np.row_stack((previous_data_features, model.means_[i]))
        
        outcome_score.append(model.score(total_data))
    # Get the index of the most probable outcome and return it
    most_probable_outcome = np.argmax(outcome_score)

    return most_probable_outcome 

In [42]:
# grab volatility and volume columns
data = raw_data[["vol", "Volume"]][10:]

# Make an HMM instance and execute fit
model = GaussianHMM(n_components=3, covariance_type="diag", n_iter=1000).fit(data)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(data)

print("Transition matrix")
print(model.transmat_)

print("Means and vars of each hidden state")
scores = []

for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covars_[i]))
    print("\n")


Transition matrix
[[9.96604087e-01 3.12318173e-03 2.72730816e-04]
 [6.23200666e-03 9.71662804e-01 2.21051896e-02]
 [2.49534296e-04 1.23920095e-02 9.87358456e-01]]
Means and vars of each hidden state
0th hidden state
mean =  [7.98438174e-01 6.62699563e+08]
var =  [1.06741606e-01 2.28734725e+17]


1th hidden state
mean =  [1.70768460e+00 4.43541608e+09]
var =  [7.87246298e-01 2.98512262e+18]


2th hidden state
mean =  [6.11919344e-01 3.38160063e+09]
var =  [5.09026676e-02 6.12753533e+17]




In [43]:
# result
res = []
for i in range(1, len(data)):
    res.append(get_most_probable_outcome(model, i))

# prediction
hmm_pred = []

# make prediction
for i in range(len(data) - 1):
    hmm_pred.append(model.means_[res[i]])

# print number of each states in the predicted sequence
print(res.count(0))
print(res.count(1))
print(res.count(2))

8254
50
0
